In [1]:
import h2o


[WARNING] H2O requires requests module of version 2.10 or newer. You have version 2.4.3.
You can upgrade to the newest version of the module running from the command line
    $ pip2 install --upgrade requests


In [3]:
h2o.connect(ip="35.196.153.55")

Connecting to H2O server at http://35.196.153.55:54321... successful.


H2O cluster uptime:,7 mins 14 secs
H2O cluster version:,3.14.0.7
H2O cluster version age:,10 days
H2O cluster name:,sparkling-water-olmsteadbrett_local-1509393572563
H2O cluster total nodes:,1
H2O cluster free memory:,2.322 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://35.196.153.55:54321
H2O connection proxy:,None


<H2OConnection to http://35.196.153.55:54321, session _sid_bae2>

In [5]:

from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [7]:
air = h2o.import_file("https://raw.github.com/0xdata/h2o/master/smalldata/airlines/allyears2k_headers.zip")

Parse progress: |█████████████████████████████████████████████████████████| 100%
